In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [10]:
# Importing the dataset
df = pd.read_csv('Churn_Modelling.csv')
X = df.iloc[:, 3:13]
y = df.iloc[:, 13]


In [11]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [12]:
#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [13]:
## Concatenate the Data Frames

X=pd.concat([X,geography,gender],axis=1)


In [14]:
## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)

In [15]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [16]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
 
 

In [17]:
sc = StandardScaler()

In [18]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test) 

### Perform Hyperparameter Optimization

In [19]:
from keras.wrappers.scikit_learn import KerasClassifier

In [20]:
from sklearn.model_selection import GridSearchCV

In [21]:
from keras.models import Sequential

In [26]:
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization,Dropout

In [27]:
from keras.activations import relu, sigmoid

In [28]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    

In [29]:
model = KerasClassifier(build_fn=create_model, verbose=0)

<ipython-input-29-7b0f05354f25>:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [30]:
layers = [[20], [40, 20], [45, 30, 15]]

In [31]:
activations = ['sigmoid', 'relu']

In [32]:
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])

In [33]:
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

In [34]:
grid_result = grid.fit(X_train, y_train)

In [35]:
[grid_result.best_score_,grid_result.best_params_]

[0.8569999933242798,
 {'activation': 'relu', 'batch_size': 128, 'epochs': 30, 'layers': [40, 20]}]